In [1]:
%matplotlib inline

In [2]:
import cv2
import torch
import matplotlib.pyplot as plt

# Load the image
image_path = 'sample_frame.jpg'
img = cv2.imread(image_path)

# Load YOLOv5n model
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)

# Perform inference
results = model(img)

# Get bounding boxes, confidence scores, and class names
results_df = results.pandas().xyxy[0]  # Get results as a Pandas DataFrame

# Draw bounding boxes on the image
for index, row in results_df.iterrows():
    x1, y1, x2, y2, conf, cls = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']), row['confidence'], row['name']
    label = f'{cls} {conf:.2f}'  # Class name and confidence
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw rectangle
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add label text

# Save the output image
output_image_path = 'output_image.jpg'
cv2.imwrite(output_image_path, img)

# Display the output image in the Jupyter notebook
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

Using cache found in C:\Users\pes12/.cache\torch\hub\ultralytics_yolov5_master
c:\Users\pes12\anaconda3\envs\env1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  2024-10-2 Python-3.12.2 torch-2.2.2 CUDA:0 (NVIDIA GeForce MX350, 2048MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


In [3]:

img = plt.imread(output_image_path)

# Display the image
plt.imshow(img)
plt.axis('off')  # Remove axes
plt.show()

In [24]:
import torch
from PIL import Image
import cv2
import numpy as np

# Load YOLOv5 model (make sure to have YOLOv5 repo cloned or installed)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Disable the post-processing (NMS, confidence thresholding, etc.)
model.eval()  # Set the model to evaluation mode

# Function to preprocess the image for YOLOv5
def preprocess_image(image_path):
    # Open the image
    img = Image.open(image_path)

    # Convert the image to RGB format (if not already)
    img = img.convert('RGB')

    # Resize the image to (640, 640) as expected by YOLOv5
    img = np.array(img)
    img = cv2.resize(img, (640, 640))

    # Convert image to tensor and normalize (YOLOv5 expects values in [0,1])
    img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0  # Normalize to 0-1
    img = img.unsqueeze(0)  # Add batch dimension

    return img

# Path to the input image
image_path = 'sample_frame.jpg'  # Replace with your image path

# Preprocess the image
input_tensor = preprocess_image(image_path)

# Forward pass through the model to get raw outputs (without post-processing)
with torch.no_grad():
    raw_output = model(input_tensor)  # This returns a list with raw output tensors

# Print the raw output vector (unprocessed)
print("Raw output (before post-processing):")
print(raw_output[0])  # Access the tensor from the list


Raw output (before post-processing):
tensor([[2.51584e+00, 4.97705e+00, 7.38330e+00,  ..., 2.39702e-03, 1.28765e-03, 3.53809e-03],
        [1.31926e+01, 3.96431e+00, 2.29371e+01,  ..., 3.56491e-03, 1.52568e-03, 4.26607e-03],
        [1.89548e+01, 3.51026e+00, 2.93802e+01,  ..., 3.77357e-03, 1.66182e-03, 5.68593e-03],
        ...,
        [5.62466e+02, 5.99950e+02, 1.72303e+02,  ..., 3.26404e-03, 8.99655e-04, 1.21293e-03],
        [5.84729e+02, 6.02651e+02, 1.22408e+02,  ..., 2.94512e-03, 9.42764e-04, 1.28025e-03],
        [6.20365e+02, 6.11995e+02, 1.61157e+02,  ..., 2.66731e-03, 1.04668e-03, 1.49785e-03]], device='cuda:0')


In [27]:
raw_output.shape,raw_output[0].shape

(torch.Size([1, 25200, 85]), torch.Size([25200, 85]))